# Read in from anupam and siphon flat materials

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from xiespp import synthesizability, get_test_samples

2024-02-29 13:59:50.103857: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
df = pd.read_csv('Compound_flatness_score.csv')
df

,Unnamed: 0,MP-ID,Comp Flatness score,Flat/not-flat,Flat segments
0,0,mp-1,0.189216,False,[]
1,1,mp-2,0.197447,False,[]
2,2,mp-3,0.350097,False,[]
3,3,mp-4,0.143728,False,[]
4,4,mp-7,0.799074,True,[4]
...,...,...,...,...,...
69814,69814,mp-1446611,0.285079,False,[]
69815,69815,mp-1475723,0.172382,False,[]
69816,69816,mp-1475733,0.922906,True,"[1, 2, 4]"
69817,69817,mp-1503976,0.130819,False,[]


In [3]:
df_slice = df[['MP-ID','Flat/not-flat']]
# extract materials with at least one flat band across the whole k-space path chosen
df_flat_only = df_slice.loc[df_slice['Flat/not-flat']==True]
df_flat_only

,MP-ID,Flat/not-flat
4,mp-7,True
18,mp-25,True
26,mp-35,True
33,mp-43,True
64,mp-83,True
...,...,...
69787,mp-1443188,True
69788,mp-1443279,True
69791,mp-1443442,True
69806,mp-1445771,True


In [4]:
flat_ids = df_flat_only['MP-ID'].to_numpy()
#print(flat_ids)
np.savetxt('flat_mat_ids', flat_ids, delimiter=',', fmt='%s')

## Using the 2nd version of the synthesizability predictor

#### Order the .CIF files in terms of rising material project-id

In [5]:
def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    '''
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]
def atoi(text):
    return int(text) if text.isdigit() else text

In [6]:
samples = get_test_samples('all_flat_mats')
samples.sort(key=natural_keys)
print(len(samples))

33399


In [7]:
model = synthesizability.SynthesizabilityPredictor()
model

<SynthesizabilityPredictor: device=/device:CPU:0, ensemble=100>

In [ ]:
from IPython.display import clear_output

sample_chunks = np.array_split(samples, 5)
scores = np.array([])

for sample_chunk in sample_chunks:
    # remove excessive warnings to prevent out-of-memory error
    clear_output(wait=True)
    
    scores = np.append(scores, model.predict(sample_chunk))
    np.savetxt('flat_material_scores.csv', scores, delimiter=',', fmt='%s')

Reading CIF files...


100%|██████████████████████████████████████| 6680/6680 [00:35<00:00, 186.71it/s]

Preparing image objects...


Checking image requirements...


 89%|█████████████████████████████████▉    | 5976/6680 [00:16<00:01, 359.05it/s]